<a href="https://www.kaggle.com/code/shreyanshg/dungeons-and-dragons-campaign-assistant?scriptVersionId=256320513" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# An Agent That Answers Dungeons & Dragons Questions with Grounded Context
Welcome to your custom Generative AI notebook, where we build an intelligent assistant that can answer natural language questions about Dungeons & Dragons adventures—powered by the Gemini API and grounded in the actual source PDFs. This notebook showcases a Retrieval-Augmented Generation (RAG) pipeline enhanced with document grounding, enabling our model to not only generate high-quality answers, but also cite the exact page and source document it drew from.



In this notebook, you will:

Load and chunk a Dungeons & Dragons adventure PDF

Store it in a Chroma vector database with metadata (page, chunk ID, and source document)

Query the database using Gemini embeddings

Generate conversational, beginner-friendly answers with visible citations


## Setup

In [2]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"
!pip install -q pymupdf

In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

### Checking for Embedding-Supported Gemini Models

Before we can embed our documents and queries, we need to verify which Gemini models support embedding.

The code below initializes the Gemini client using your API key and lists all available models that support the embedContent action. This ensures you're selecting a model that's compatible with vector embedding for use in your RAG pipeline.

We’ll be using one of these models to generate vector embeddings for both our Dungeons & Dragons document chunks and user questions.

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001


### Data

Here is the documents you will use to create an embedding database.

In [6]:
import os
import fitz  # PyMuPDF

# List of all PDFs you want to index
pdf_paths = [
    "/kaggle/input/dnd-northern-palace/DRA17_northernpalace.pdf",
    "/kaggle/input/dnd-file-oneshots/5E_Wolves_Of_Welton.pdf",
    "/kaggle/input/dnd-file-oneshots/the_wild_sheep_chase_v2.pdf"
]

chunks = []
chunk_id_counter = 0

for pdf_path in pdf_paths:
    source_doc = os.path.basename(pdf_path)
    
    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"Failed to open {source_doc}: {e}")
        continue  # skip to next doc

    for i, page in enumerate(doc):
        page_text = page.get_text()
        page_text = page_text.replace('\xa0', ' ').strip()

        for section in page_text.split('\n\n'):
            clean = section.strip()
            if len(clean) > 30:
                chunks.append({
                    "text": clean,
                    "page": i + 1,
                    "source_doc": source_doc,
                    "chunk_id": chunk_id_counter
                })
                chunk_id_counter += 1

# Prepare data for ChromaDB
documents = [chunk["text"] for chunk in chunks]
metadatas = [{"page": chunk["page"], "chunk_id": chunk["chunk_id"], "source_doc": chunk["source_doc"]}
             for chunk in chunks]
ids = [str(chunk["chunk_id"]) for chunk in chunks]

# Preview a sample chunk
print(metadatas[0])




{'page': 1, 'chunk_id': 0, 'source_doc': 'DRA17_northernpalace.pdf'}


## Creating the embedding database with ChromaDB

Create a [custom function](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) to generate embeddings with the Gemini API. In this task, you are implementing a retrieval system, so the `task_type` for generating the *document* embeddings is `retrieval_document`. Later, you will use `retrieval_query` for the *query* embeddings. Check out the [API reference](https://ai.google.dev/api/embeddings#v1beta.TaskType) for the full list of supported tasks.

Key words: Documents are the items that are in the database. They are inserted first, and later retrieved. Queries are the textual search terms and can be simple keywords or textual descriptions of the desired documents.

In [7]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

Now create a [Chroma database client](https://docs.trychroma.com/getting-started) that uses the `GeminiEmbeddingFunction` and populate the database with the documents you defined above.

In [8]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)


Confirm that the data was inserted by looking at the database.

In [9]:
db.count()
# You can peek at the data too.
#db.peek(1)

25

## Retrieval: Find relevant documents

To search the Chroma database, call the `query` method. Note that you also switch to the `retrieval_query` mode of embedding generation.


In [10]:
def retrieve_passages(query, k=5, topic_hint=None):
    embed_fn.document_mode = False
    result = db.query(query_texts=[query], n_results=k * 4)  # Get more to allow filtering
    all_passages = result["documents"][0]
    all_metadatas = result["metadatas"][0]

    # Normalize topic hint for comparison
    def matches_topic(meta, hint):
        source = meta.get("source_doc", "").lower()
        return hint.lower() in source

    # Apply topic filtering
    if topic_hint:
        filtered = [(p, m) for p, m in zip(all_passages, all_metadatas) if matches_topic(m, topic_hint)]

        if filtered:
            top_k = filtered[:k]
            all_passages, all_metadatas = zip(*top_k)
        else:
            # No match found, fallback to unfiltered top-k
            all_passages, all_metadatas = all_passages[:k], all_metadatas[:k]
    else:
        # No topic hint, return top-k directly
        all_passages, all_metadatas = all_passages[:k], all_metadatas[:k]

    return list(all_passages), list(all_metadatas)


## Augmented generation: Answer the question

Now that you have found a relevant passage from the set of documents (the *retrieval* step), you can now assemble a generation prompt to have the Gemini API *generate* a final answer. Note that in this example only a single passage was retrieved. In practice, especially when the size of your underlying data is large, you will want to retrieve more than one result and let the Gemini model determine what passages are relevant in answering the question. For this reason it's OK if some retrieved passages are not directly related to the question - this generation step should ignore them.

In [11]:
from collections import defaultdict

def build_prompt(query, retrieved_passages, metadatas):
    query_oneline = query.replace("\n", " ")

    prompt = f"""
You are a knowledgeable and friendly Dungeons & Dragons assistant. Use only the reference passages provided below to answer the question. 
If the answer is not clearly supported by the passages, say you’re not sure rather than making something up.

Be detailed, but use a conversational tone — imagine you're helping a new player or Dungeon Master. 
Explain any terms that may be unclear to beginners, and try to include the context where appropriate.

Here are some examples:

---

QUESTION: What is the Challenge Rating (CR) of a Beholder?

REFERENCE PASSAGE: A Beholder is a terrifying aberration with a Challenge Rating (CR) of 13. It can fly and shoot devastating eye rays.

ANSWER: A Beholder has a Challenge Rating of 13. This means it’s a very dangerous monster—powerful enough to challenge an entire party of adventurers.

---

QUESTION: What is a spell slot?

REFERENCE PASSAGE: Spell slots represent the number of spells a caster can use per day. When a spell is cast, it consumes a slot of its level or higher.

ANSWER: A spell slot is a resource that limits how many spells a magic user can cast. Each time a spell is used, one of the available slots is used up. Stronger spells require higher-level slots.

---

Now answer the following question:

QUESTION: {query_oneline}
"""

    # Group by document
    grouped_passages = defaultdict(list)
    for passage, meta in zip(retrieved_passages, metadatas):
        doc = meta.get("source_doc", "Unknown Document")
        grouped_passages[doc].append((meta.get("page"), passage))

    for doc_name, entries in grouped_passages.items():
        for page, passage in entries:
            passage_oneline = passage.replace("\n", " ").strip()
            if len(passage_oneline) > 1000:
                passage_oneline = passage_oneline[:1000] + "..."
            prompt += f"\nREFERENCE PASSAGE ({doc_name}, Page {page}): {passage_oneline}"

    prompt += "\n\nNow, using only the reference passages, write your answer below:\n\nANSWER:"
    return prompt

Now use the `generate_content` method to to generate an answer to the question.

In [12]:
from IPython.display import Markdown

def generate_and_display_answer(prompt, metadatas):
    # Generate answer using Gemini
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    answer = response.text.strip()

    # Deduplicate and format sources
    seen = set()
    source_info_lines = []
    for meta in metadatas:
        key = (meta.get("source_doc"), meta.get("page"), meta.get("chunk_id"))
        if key not in seen:
            seen.add(key)
            source_info_lines.append(
                f"- {meta.get('source_doc')} (Page {meta.get('page')}, Chunk {meta.get('chunk_id')})"
            )

    source_info = "\n".join(source_info_lines)

    # Display result
    final_output = f"{answer}\n\n---\n**Sources:**\n{source_info}"
    display(Markdown(final_output))

    return answer, source_info


In [13]:
import re
import json
from IPython.display import Markdown

# === Campaign Keywords ===
campaign_keywords = {
    "wolves of welton": "wolves",
    "nicholas": "nicholas",
    "northern palace": "palace",
    "wild sheep chase": "sheep",
    "sheep": "sheep"
}

# === Fuzzy Topic Detection ===
def detect_topic(query, known_campaigns, current_topic=None):
    query = query.lower()
    
    # Direct override (e.g. "focus on Nicholas")
    match = re.search(r"focus on (.+)", query)
    if match:
        requested = match.group(1).strip()
        for phrase, topic_key in known_campaigns.items():
            if re.search(rf"\b{re.escape(phrase)}\b", requested):
                return topic_key
    
    # Only trigger if no topic is set or user says something like "let's talk about"
    if current_topic is None or any(p in query for p in ["let's talk about", "tell me about", "start with"]):
        for phrase, topic_key in known_campaigns.items():
            if re.search(rf"\b{re.escape(phrase)}\b", query):
                return topic_key
    
    return current_topic  # Default: preserve current topic

def run_chat_assistant():
    global chat_history, current_topic
    if "chat_history" not in globals():
        chat_history = []
    if "current_topic" not in globals():
        current_topic = None

    print("🧙 Welcome to the D&D Assistant!")
    print("You can ask questions about adventures or use the following commands:")
    print("• @summarize <topic>   — summarize past answers about a topic")
    print("• @rerun <k>           — re-run last query with k chunks")
    print("• @truncate <n>        — keep last n messages")
    print("• @save <file.json>    — save chat history")
    print("• @reload <file.json>  — reload chat history")
    print("• @campaigns           — list available adventure PDFs")
    print("• clear topic / reset topic / exit\n")

    while True:
        query = input("💬 You: ").strip()

        # === Exit
        if query.lower() == "exit":
            print("🧝‍♂️ Assistant: Farewell, adventurer!")
            break

        # === Clear topic
        if query.lower() in {"clear topic", "reset topic"}:
            current_topic = None
            print("🧹 Topic has been cleared.")
            continue

        # === Function-calling agent commands
        if query.startswith("@") and route_agent_command(query):
            continue

        # === Normal RAG assistant flow
        chat_history.append({"role": "user", "content": query})

        # Detect topic
        new_topic = detect_topic(query, campaign_keywords, current_topic)
        if new_topic != current_topic:
            current_topic = new_topic
            print(f"📌 Topic set to: {current_topic.title() if current_topic else 'None'}")

        print(f"\n📚 Active Topic: {current_topic.title() if current_topic else 'None'}")

        # RAG steps
        retrieved_passages, metadatas = retrieve_passages(query, k=15, topic_hint=current_topic)
        prompt = build_prompt(query, retrieved_passages, metadatas)
        answer, source_info = generate_and_display_answer(prompt, metadatas)
        
        # Store in chat history silently
        chat_history.append({
            "role": "assistant",
            "content": answer,
            "sources": source_info
        })
        
        # Only show the latest response visibly
        print(f"\n🧝‍♂️ Assistant:\n{answer.strip()}\n")
        print(f"📎 Sources:\n{source_info.strip()}\n")


# === Save chat history anytime
def save_chat_history(filename="chat_history.json"):
    with open(filename, "w") as f:
        json.dump(chat_history, f, indent=2)
    print(f"💾 Chat history saved to {filename}")


In [14]:
#run_chat_assistant()

In [15]:
# from collections import Counter
# sources = [meta["source_doc"] for meta in metadatas]
# print(Counter(sources))